# Practice Session 6: Non-Normally Distributed Data and Non-Parametric Tests

In [ ]:
# Install the outlier_utils package
# We will use it in today's practice session to perform the Grubbs' test for detecting outliers
!pip install outlier_utils


In [110]:
import pandas as pd
import numpy as np

from scipy import stats
from scipy.stats import ttest_1samp, ttest_ind, ttest_rel, f, shapiro, levene, wilcoxon, mannwhitneyu
from outliers import smirnov_grubbs as grubbs

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (8, 8)
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

## Part 1: Understanding Non-Normal Data and Outliers

So far, all the hypothesis tests we have used assumed that the data were **normally distributed**.  
But what if that assumption does not hold?

There are many reasons why data might **not follow a normal distribution**.\
As we already saw in **Practice Session 2**, environmental data often follow **log-normal**, **exponential**, or other **skewed** distributions.\
In some cases, a **data transformation** (for example, log-transform) can make the data approximately normal.  

However, non-normality can also arise from the presence of **outliers**, which can introduce or increase skewness in the data.

**A quick reminder: what is skewness?**\
**Skewness** measures how asymmetric a distribution is relative to the mean:
- A **positive skew** (right-skewed) means a longer tail on the **right** side (e.g., most values are small, but a few are much larger).  
- A **negative skew** (left-skewed) means a longer tail on the **left** side (e.g., most values are large, but a few are much smaller).  

In **Homework 1**, you already calculated the **sample skewness** and **excess kurtosis** of your dataset.

**How much skewness or kurtosis is "acceptable"?**\
In practice, **small deviations from normality are usually not a problem** - most parametric tests, such as the *t*-test, are fairly robust to mild non-normality.  

Below are commonly used **rule-of-thumb thresholds** for interpreting **skewness**:
| Distribution shape | Skewness range | Interpretation |
|:---|:---:|:---|
| Approximately normal | −0.5 to +0.5 | Distribution is close to symmetric |
| Moderately skewed | −1 to −0.5 or +0.5 to +1 | Noticeable asymmetry |
| Highly skewed | ≤ −1 or ≥ +1 | Strong asymmetry, non-normal |

For **excess kurtosis**, values between **−0.5 and +0.5** are typically considered *close to normal* (that is, the distribution’s tails and peak resemble those of a normal curve.)
<div style="display: flex; align-items: center; justify-content: space-around;">
  <img src="https://raw.githubusercontent.com/RaHub4AI/MI7032/refs/heads/main/Pictures/skew.png" alt="Skewness illustration" width="48%">
  <img src="https://raw.githubusercontent.com/RaHub4AI/MI7032/refs/heads/main/Pictures/kurtosis.png" alt="Kurtosis illustration" width="48%">
</div>

**A note on outliers**\
On boxplots, potential outliers are displayed as **individual points**.  
They are identified when they fall more than **1.5 × IQR (interquartile range)** below the first quartile (Q1) or above the third quartile (Q3).\
This is a **visual guideline** (a quick way to spot unusual values) but it is **not a formal statistical test** for identifying outliers.


#### The Grubbs’ test for outliers
If we want to formally test whether a single observation is an outlier, we can use the **Grubbs’ test** (also called the **maximum normalized residual test**).\
The test detects **one outlier at a time** by comparing the largest deviation from the mean to the overall standard deviation: $G = \frac{\max |x_i - \bar{x}|}{s}$, where:
- $\bar{x}$ is the sample mean,  
- $s$ is the sample standard deviation, and  
- $x_i$ is an observation in the dataset.

The Grubbs’ test checks the following hypotheses:
- **Null hypothesis ($H_0$)**: There are **no outliers** in the data.  
- **Alternative hypothesis ($H_1$)**: There is **at least one outlier**.

If the computed test statistic $G$ exceeds the critical value (based on the *t*-distribution), we reject $H_0$ and conclude that the most extreme value is a statistical outlier.

> *Note:* Grubbs’ test assumes **normality** of the underlying data.


#### <font color="#fc7202">Task 1:</font>
You are given a 1D dataset (see the code cell below).  

1) **Visualize** the data (boxplot, histogram, Q-Q plot).  
2) **Describe** the data (count, mean, median, std, skewness, excess kurtosis).  
3) If you **suspect outliers** from the visuals, apply the **Grubbs’ test** (one outlier at a time).  
   - If an outlier is detected, **remove it**, then **repeat** the descriptive stats and visuals on the cleaned data.  
4) Briefly **summarize** what changed after removing outliers.

*Hint:* Use the `grubbs()` function from the **`outliers.smirnov_grubbs`** module (already imported for you as `grubbs.test()`) to perform the **Grubbs’ test** for detecting statistical outliers.

In [25]:
dataset_outliers = pd.DataFrame({'measurement_id': np.arange(1, 21),
                                 'observed_value': [0.4, 1.7, 0.6, 74.9, 13.9, 0.8, 20.2, 143.2, 725.7, 0.2, 160.2, 14.7, 5.0, 2.9, 171.7, 20.4, 1.4, 17.1, 79.7, 27.6]})

In [ ]:
# YOUR CODE HERE!

#### <font color="#fc7202">Task 2:</font>

A file named **`three_datasets.tsv`** contains (indeed) three different datasets.  
Exactly **one** of them is **already (approximately) normally distributed**.  
For the **other two**, your task is to find a **monotonic transformation** that makes them **approximately normal** — **without removing any data points** (no outlier trimming).

**Your goals:**
1) **Import** the data into Python.  
2) **Compute descriptive statistics** for each dataset (n, mean, median, sd, skewness, excess kurtosis, quartiles).  
3) **Visualize** each dataset (boxplot, histogram, Q-Q plot).  
4) **Identify** which dataset is already normal (use Q-Q plots + formal tests).  
5) For the **non-normal** datasets, **apply a suitable transformation**.  
6) **Re-evaluate normality** on the transformed data (Q-Q plots + Shapiro–Wilk).  
7) **Report your evidence**: show before/after plots and test *p*-values, and **justify** the chosen transformation (briefly).

In [ ]:
# YOUR CODE HERE!

## Part 2: Non-Parametric Tests

<div style="display: flex; align-items: flex-start; justify-content: space-between;">
  <div style="flex: 1; padding-right: 10px;">

Up to this point, we have used **parametric tests** such as the *t*-test and *F*-test, but we have not yet defined what actually makes a test *parametric*.

Parametric tests are based on specific **assumptions about the population**, most importantly that the data are **normally distributed** and that the **variances are similar** between groups.  
They also rely on **population parameters**, such as the **mean (μ)** and **standard deviation (σ)**, which is where the term *parametric* originates.

When these assumptions are met, parametric tests are generally **more powerful** because they make efficient use of all the data.  
However, if the data deviate strongly from normality, have unequal variances, or contain outliers, applying a parametric test can lead to **misleading conclusions**.

In such cases, it is better to use a **non-parametric test**.  
Non-parametric tests, sometimes called **distribution-free methods**, do not rely on strong assumptions about the data’s underlying distribution.  
They work on **ranks or medians** rather than means and are therefore **more robust** against skewed data and outliers, although typically **less sensitive** when the data are perfectly normal.

In practice, the goal is not to avoid parametric tests but to **choose the method that best fits the data**.  
If your dataset reasonably meets the assumptions, use a parametric test.  
If not, a non-parametric test provides a safer and more reliable alternative.

The diagram on the right summarizes this general decision process:  
when to stay within the parametric framework and when to switch to non-parametric approaches.

  </div>
  <div style="flex: 0 0 50%; text-align: right;">
    <img src="https://raw.githubusercontent.com/RaHub4AI/MI7032/refs/heads/main/Pictures/normal_nonnormal_data.png"
         alt="Workflow: choosing parametric vs. non-parametric tests"
         width="95%"
         style="border-radius: 8px;">
  </div>
</div>

In today’s session, we will focus on two widely used **non-parametric tests** that serve as direct alternatives to the *t*-tests we have used before:
- The **Wilcoxon Signed-Rank Test**, which is the non-parametric alternative to the **one-sample** *t*-test or the **paired two-sample** *t*-test.  
- The **Mann-Whitney U Test**, which replaces the **independent two-sample** *t*-test.  




#### The Wilcoxon Signed-Rank Test

The **Wilcoxon Signed-Rank Test** is used when we have **paired data** (two related samples) or when we want to test whether the **median** of a single sample differs from a hypothesized value.  

**How it works:**
1. For each observation, calculate the **difference** between the two paired values (or between the observed value and the hypothesized median).  
2. Ignore any pairs with a difference of 0.  
3. Take the **absolute values** of the differences and **rank** them from smallest to largest.  
4. Assign the **sign** of the original difference (+ or −) to each rank.  
5. Sum the ranks for positive ($W^+$) and negative ($W^-$) differences separately.  
6. The **test statistic** is often $W = \min(W^+, W^-)$ (i.e., the smaller of the two signed rank sums).
7. Under the null hypothesis (no median difference), the positive and negative ranks should be **balanced**.  
   For small samples, *critical values* of $W$ are tabulated.  
   For larger samples ($n > 10$), $W$ approximately follows a **normal distribution** with  $\mu_W = \frac{n(n+1)}{4}, \quad \sigma_W = \sqrt{\frac{n(n+1)(2n+1)}{24}}$\
   The standardized test statistic is then $Z = \frac{W - \mu_W}{\sigma_W}$ which can be compared to the standard normal distribution to obtain the *p*-value.

If the data come from a symmetric distribution centered around zero, positive and negative ranks balance out.  
A large imbalance between $W^+$ and $W^-$ indicates a systematic shift - evidence against $H_0$.

**Hypotheses:**

- $H_0$: The median difference between pairs is zero.  
- $H_1$: The median difference is not zero (two-tailed), or greater/less than zero (one-tailed).

---

**Example:**\
Let $X$ denote the length (in cm) of a randomly selected pygmy sunfish.\
Given the sample: $\{\,5.0,\; 3.9,\; 5.2,\; 5.5,\; 2.8,\; 6.1,\; 6.4,\; 2.6,\; 1.7,\; 4.3\,\}$ test whether the **population median** differs from **3.7 cm** at the 95% of confidence level.

*Hypotheses:*
- $H_0:\ \text{median}(X)=3.7$
- $H_1:\ \text{median}(X)\neq 3.7$

*Test choice:* Wilcoxon **signed-rank** test (one-sample, two-sided).  

<div style="display: flex; align-items: flex-start; justify-content: space-between;">
  <div style="flex: 1; padding-right: 10px;">

*Manual Calculation*

1. Compute the differences from the hypothesized median (3.7 cm)

| Observation | Value (cm) | Difference (x − 3.7) |
|:--:|--:|--:|
| 1 | 5.0 | +1.3 |
| 2 | 3.9 | +0.2 |
| 3 | 5.2 | +1.5 |
| 4 | 5.5 | +1.8 |
| 5 | 2.8 | −0.9 |
| 6 | 6.1 | +2.4 |
| 7 | 6.4 | +2.7 |
| 8 | 2.6 | −1.1 |
| 9 | 1.7 | −2.0 |
| 10 | 4.3 | +0.6 |

No zero differences, so $n = 10$.

2. Take absolute differences and rank them (smallest = 1)

| Observation | Difference | abs(Difference) | Rank | Signed Rank |
|:--:|--:|--:|--:|--:|
| 2 | +0.2 | 0.2 | 1 | +1 |
| 10 | +0.6 | 0.6 | 2 | +2 |
| 5 | −0.9 | 0.9 | 3 | −3 |
| 8 | −1.1 | 1.1 | 4 | −4 |
| 1 | +1.3 | 1.3 | 5 | +5 |
| 3 | +1.5 | 1.5 | 6 | +6 |
| 4 | +1.8 | 1.8 | 7 | +7 |
| 9 | −2.0 | 2.0 | 8 | −8 |
| 6 | +2.4 | 2.4 | 9 | +9 |
| 7 | +2.7 | 2.7 | 10 | +10 |

  </div>
  <div style="flex: 0 0 60%; text-align: left;">
    <img src="https://raw.githubusercontent.com/RaHub4AI/MI7032/refs/heads/main/Pictures/fish.png"
         alt="Workflow: choosing parametric vs. non-parametric tests"
         width="55%"
         style="border-radius: 8px;">
  </div>
</div>


3. Sum the positive and negative ranks
    - $W^+ = 1 + 2 + 5 + 6 + 7 + 9 + 10 = 40$
    - $W^- = 3 + 4 + 8 = 15$

4. Test statistic\
    $W = \min(W^+, W^-) = 15$

5. Decision\
For $n = 10$, two-sided critical region (α = 0.05) ≈ $W \leq 8$. (https://users.stat.ufl.edu/~winner/tables/wilcox_signrank.pdf)\
Since $W = 15 > 8$, we **fail to reject $H_0$**.

*Conclusion:* There is **no statistically significant evidence** that the median pygmy sunfish length differs from **3.7 cm**.  
Hence, the **median length ≈ 3.7 cm** is consistent with the sample data.




In [107]:
# Same test in Python using stats.wilcoxon()
x = np.array([5.0, 3.9, 5.2, 5.5, 2.8, 6.1, 6.4, 2.6, 1.7, 4.3])

# one-sample Wilcoxon signed-rank against median = 3.7
res = wilcoxon(x - 3.7, alternative='two-sided', zero_method='wilcox', method='exact')
print(f"W = {res.statistic:.0f}, p = {res.pvalue:.4f}")

W = 15, p = 0.2324


> *Conclusion (*α* = 0.05):* $p > 0.05$ → fail to reject $H_0$\
> There is no statistically significant evidence that the median pygmy sunfish length differs from 3.7 cm.

#### The Mann-Whitney *U* Test

The **Mann-Whitney *U* Test** (also known as the **Wilcoxon Rank-Sum Test**) compares **two independent samples** and tests whether one tends to have larger values than the other.  
It serves as the **non-parametric counterpart** to the *independent two-sample t-test*.

**How it works:**
1. Combine all observations from both groups into one dataset.  
2. Rank all values together from smallest to largest (assign average ranks for ties).  
3. Compute the **sum of ranks** for each group.  
4. The ***U* statistic** measures how much the ranks from one group differ from what would be expected under the null hypothesis of identical distributions.

If the distributions are truly the same, the ranks should be evenly mixed.  
If one group tends to have systematically higher or lower ranks, the test statistic will reflect this imbalance.

**Hypotheses:**
- $H_0$: The two groups come from the same population (distributions are identical).  
- $H_1$: The distributions differ (e.g., one tends to yield larger values).

---

**Example**

We have independent samples of **body mass (in grams)** from two lizard species:
- **Geckos:** 3.186, 2.427, 4.031, 1.995  
- **Anoles:** 5.515, 5.659, 6.739, 3.184  

We want to test whether the **two species differ in body mass**, without assuming normality and without specifying a direction of difference.

*Hypotheses:*
- $H_0$: The two species have the same population distribution (no difference in medians)
- $H_1$: The body mass distributions differ between Geckos and Anoles

Because the data are **not normally distributed** and the sample sizes are **small (*n* = 4 per group)**,  
we use the **Mann–Whitney U test**, a **non-parametric alternative** to the independent two-sample *t*-test.

*Manual calculation*

<div style="display: flex; align-items: flex-start; justify-content: space-between;">
  <div style="flex: 1; padding-right: 10px;">

1. Pool all observations and rank from smallest to largest (no ties here):

| Value | Group | Rank |
|:-----:|:----:|:---:|
| 1.995 | G | 1 |
| 2.427 | G | 2 |
| 3.184 | A | 3 |
| 3.186 | G | 4 |
| 4.031 | G | 5 |
| 5.515 | A | 6 |
| 5.659 | A | 7 |
| 6.739 | A | 8 |

2. Sum of ranks  
- Geckos: $R_G = 1 + 2 + 4 + 5 = 12$  
- Anoles: $R_A = 3 + 6 + 7 + 8 = 24$

3. U statistics \
$(n_G = n_A = 4)$\
$U_G = R_G - \frac{n_G(n_G+1)}{2} = 12 - 10 = 2$\
$U_A = R_A - \frac{n_A(n_A+1)}{2} = 24 - 10 = 14$

    The test statistic is: $U = \min(U_G, U_A) = \mathbf{2}$

  </div>
  <div style="flex: 0 0 60%; text-align: left;">
    <img src="https://raw.githubusercontent.com/RaHub4AI/MI7032/refs/heads/main/Pictures/geckos_anoles.png"
         alt="Workflow: choosing parametric vs. non-parametric tests"
         width="55%"
         style="border-radius: 8px;">
  </div>
</div>

4. Decision (two-sided, α = 0.05):  
For $n_1=n_2=4$, the **critical value** is $U_crit = 0$ (https://media.geeksforgeeks.org/wp-content/uploads/20201110022047/mannwhitneytwotailed.PNG)\
Since our observed value $U=2$ is greater than the critical value, we fail to reject the null hypothesis.\
There is no statistically significant difference between the two groups at the 5% significance level.



In [111]:
geckos = np.array([3.186, 2.427, 4.031, 1.995])
anoles = np.array([5.515, 5.659, 6.739, 3.184])

# Two-sided, exact method for small samples
res = mannwhitneyu(geckos, anoles, alternative='two-sided', method='exact')
print(f"U = {res.statistic}, p-value = {res.pvalue:.4f}")

U = 2.0, p-value = 0.1143


> *Conclusion:* Since this *p*-value is greater than the significance level (*α* = 0.05), we fail to reject the null hypothesis.
There is no statistically significant difference in the median body mass between geckos and anoles.

#### <font color="#fc7202">Task 3:</font>

The lead concentration in the blood (pg/mL) of seven children from Stockholm was measured as follows:
> 104, 79, 98, 150, 87, 136, 101

We want to know whether these data differ significantly from the Swedish population median, which is reported to be 95 pg/mL.

Your tasks:
1. Formulate the hypotheses clearly.
2. Apply both a Wilcoxon signed-rank test and a one-sample *t*-test to evaluate whether the Stockholm sample differs significantly from the population median (or mean).
3. Compare and interpret the results of both tests.


In [ ]:
# YOUR CODE HERE!